# COVID-19 Infection Risk Prediction with Amazon SageMaker Autopilot

---

## Contents

1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#SetupAPjob)
1. [Autopilot Results](#Results)
1. [Batch Inference](#BatchInf)
1. [Cleanup](#Cleanup)


---

## Introduction

Amazon SageMaker Autopilot is an automated machine learning (commonly referred to as AutoML) solution for tabular datasets. You can use SageMaker Autopilot in different ways: on autopilot (hence the name) or with human guidance, without code through SageMaker Studio, or using the AWS SDKs. This notebook, as a first glimpse, will use the AWS SDKs to simply create and deploy a machine learning model.

Coronavirus disease (COVID-19) is an infectious disease caused by a new virus. People can catch COVID-19 from others who have the virus. While we are still learning about how COVID-19 affects people, older persons and persons with pre-existing medical conditions (such as high blood pressure, heart disease, lung disease, cancer or diabetes) appear to develop serious illness more often than others.

In this notebook we are going to demonstrate how we can use AutoPilot to create a model to predict COVID-19 infection risk based on various factors like age, sex, country, present medical conditions, number of people contacted with, steps taken by the government to contain the risk, etc.   

---
## Setup


Import the required libraries

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import pandas as pd 
import random
import json
import math
import time
import datetime
import s3fs
import boto3
import sagemaker
import io
from datetime import timedelta
import sagemaker
import boto3
from sagemaker import get_execution_role

The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting. The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [ ]:
region = boto3.Session().region_name

session = sagemaker.Session()

# You can modify the following to use a bucket of your choosing
bucket = session.default_bucket()
prefix = 'sagemaker/DEMO-autopilot-covid19'

role = get_execution_role()

# This is the client we will use to interact with SageMaker AutoPilot
sm = boto3.Session().client(service_name='sagemaker',region_name=region)

## Data

We are going to use the data which is generated by a public survey which captures various data like age, sex, country, health conditions and other relevant data along with labels like probability of covid-19 infection and mortality. In this example we are going to use probability of covid-19 infection (column name: risk_infection) as our target column.

The dataset we use is publicly available at https://www.covid19survivalcalculator.com/download. 

In [ ]:
#download the csv file to local
!wget https://www.covid19survivalcalculator.com/data/master_dataset.csv

In [ ]:
# capture only the columns 
df = pd.read_csv('master_dataset.csv', nrows=0)
tempheaderdf = pd.DataFrame(df)
tempheaderdf.columns

In [ ]:
# read the rows, assign column names to the data and drop columns we are not going to use in this notebook 
#(you can modify this notebook to suit your needs)
df = pd.read_csv('master_dataset.csv', parse_dates=True, header=None, skiprows=[0])
pd.set_option('display.max_columns', 50)
df.drop(df.columns[[43]], axis = 1, inplace = True)
df.columns = tempheaderdf.columns
df.drop(['survey_date', 'region', 'ip_accuracy', 'prescription_medication', 'risk_mortality'], axis = 1, inplace = True)

Following are the columns we are going to use for this notebook:
- `country`: two-letter abbreviation for country; for example, US or GB
- `ip_latitude`: IP location latitude randomly adjusted (-0.1 to +0.1) as per the download website
- `ip_longitude`: IP location longitude randomly adjusted (-0.1 to +0.1) as per the download website
- `sex`: Sex of the individual
- `age`: Age of the individual reduced to 10-year bandings as per the download website
- `height`: Height of the individual
- `weight`: Weight of the individual
- `bmi`: BMI (Body Mass Index) of the individual
- `blood_type`: Blood group
- `smoking`: Does the individual smoke or not
- `alcohol`: Does the individual consume alcohol or not
- `cannabis`: Does the individual use Non-Prescription Drugs (cannabis) or not
- `amphetamines`: Does the individual use Non-Prescription Drugs (amphetamines) or not
- `cocaine`: Does the individual use Non-Prescription Drugs (cocaine) or not
- `lsd`: Does the individual use Non-Prescription Drugs (lsd) or not
- `mdma`: Does the individual use Non-Prescription Drugs (mdma) or not
- `contacts_count`: How many people was the individual in close contact with in the last week? (including partners, children, work colleagues, customers, patients etc.)
- `house_count`: How many people live in individual's house / flat
- `text_working`: Are traveling to work / school?
- `rate_government_action`: Is the individual's country's government taking COVID-19 seriously
- `rate_reducing_risk_single`: Is the individual taking steps to reduce his/her risk (social distancing, washing hands).
- `rate_reducing_risk_house`: Do the people the individual live with are taking steps to reduce risk (social distancing, washing hands).
- `rate_reducing_mask`: Does the individual wear a mask when outside of house / flat?
- `covid19_positive`: Is the individual covid19 positive
- `covid19_symptoms`: Does the individual have covid19 symptoms (dry cough, fever, breathlessness etc)
- `covid19_contact`: Is the individual come in contact with a person having covid19
- `asthma`: Does the individual have asthma
- `kidney_disease`: Does the individual have kidney disease
- `compromised_immune`: Does the individual have compromised immune system
- `heart_disease`: Does the individual have heart disease
- `lung_disease`: Does the individual have lung disease
- `diabetes`: Does the individual have diabetes
- `hiv_positive`: Is the individual hiv positive
- `hypertension`: Does the individual have hypertension
- `other_chronic`: Does the individual have any other chronic
- `opinion_infection`: Individual's own opinion of his/her chances of getting COVID-19
- `opinion_mortality`: Individual's own opinion of his/her chances of dying if they have COVID-19
- `risk_infection`: Probaility of getting infected with COVID-19 (<b>Target Attribute/Column</b>)


In [ ]:
df.head()

In [ ]:
df.shape

Analyze the distribution of the data for the target column

In [ ]:
df['risk_infection'].describe()

In [ ]:
axarr = df.hist(column='risk_infection', figsize=(15,5))
for ax in axarr.flatten():
    ax.set_xlabel("Risk Infection Probability")
    ax.set_ylabel("Records in the dataset")

### Preparing data for training and testing

Divide the data into training and testing splits. The training split is used by SageMaker Autopilot. The testing split will be used for inference.

In [ ]:
train_data = df.sample(frac=0.8,random_state=200)
test_data = df.drop(train_data.index)
#Drop the target column from the test data
test_data_no_target = test_data.drop(columns=['risk_infection'])

print('Total Record Count of the Dataset: ' + str(df.shape[0]))
print('Training Data Record Count: ' + str(train_data.shape[0]))
print('Testing Data Record Count: ' + str(test_data.shape[0]))

Upload the training and test files to S3 locations

In [ ]:
train_file = 'train_data.csv';
train_data.to_csv(train_file, index=False, header=True)
train_data_s3_path = session.upload_data(path=train_file, key_prefix=prefix + "/train")
print('Train data uploaded to: ' + train_data_s3_path)

test_file = 'test_data.csv';
test_data_no_target.to_csv(test_file, index=False, header=False)
test_data_s3_path = session.upload_data(path=test_file, key_prefix=prefix + "/test")
print('Test data uploaded to: ' + test_data_s3_path)

## Setting up the SageMaker Autopilot Job<a name="SetupAPjob"></a>

After uploading the dataset to Amazon S3, you can invoke Autopilot to find the best ML pipeline to train a model on this dataset. 

The required inputs for invoking a Autopilot job are:
* Amazon S3 location for input dataset and for all output artifacts
* Name of the column of the dataset you want to predict (`risk_infection` in this case) 
* An IAM role

Currently Autopilot supports only tabular datasets in CSV format. Either all files should have a header row, or the first file of the dataset, when sorted in alphabetical/lexical order by name, is expected to have a header row.

You can also specify the type of problem you want to solve with your dataset (`Regression, MulticlassClassification, BinaryClassification`). In case you are not sure, SageMaker Autopilot will infer the problem type based on statistics of the target column (the column you want to predict). 

Because the target attribute, ```risk_infection```, is a real value, our model will be performing regression. In this example we will let AutoPilot infer the type of problem for us.

You have the option to limit the running time of a SageMaker Autopilot job by providing either the maximum number of pipeline evaluations or candidates (one pipeline evaluation is called a `Candidate` because it generates a candidate model) or providing the total time allocated for the overall Autopilot job. Under default settings, this job takes about four hours to run. This varies between runs because of the nature of the exploratory process Autopilot uses to find optimal training parameters.

In [ ]:
input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': 's3://{}/{}/train'.format(bucket,prefix)
        }
      },
      'TargetAttributeName': 'risk_infection'
    }
  ]

output_data_config = {
    'S3OutputPath': 's3://{}/{}/output'.format(bucket,prefix)
  }

### Launching the SageMaker Autopilot Job

You can now launch the Autopilot job by calling the `create_auto_ml_job` API. We limit the number of candidates to 20 so that the job finishes in a few minutes.

In [ ]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-covid19-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      AutoMLJobConfig={'CompletionCriteria':
                                       {'MaxCandidates': 20}
                                      },
                      RoleArn=role)

### Tracking SageMaker Autopilot job progress
SageMaker Autopilot job consists of the following high-level steps : 
* Analyzing Data, where the dataset is analyzed and Autopilot comes up with a list of ML pipelines that should be tried out on the dataset. The dataset is also split into train and validation sets.
* Feature Engineering, where Autopilot performs feature transformation on individual features of the dataset as well as at an aggregate level.
* Model Tuning, where the top performing pipeline is selected along with the optimal hyperparameters for the training algorithm (the last stage of the pipeline). 

In [ ]:
print ('JobStatus - Secondary Status')
print('------------------------------')


describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
job_run_status = describe_response['AutoMLJobStatus']
    
while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response['AutoMLJobStatus']
    
    print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
    sleep(30)

---
## Results

Use the `describe_auto_ml_job` API to look up the best candidate selected by the SageMaker Autopilot job based on the objective metric selected by SageMaker AutoPilot

In [ ]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_name = best_candidate['CandidateName']
print(best_candidate)
print('\n')
print("CandidateName: " + best_candidate_name)
print("FinalAutoMLJobObjectiveMetricName: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("FinalAutoMLJobObjectiveMetricValue: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

---
## Batch inference using the best candidate<a name="BatchInf"></a>

Create a model using the best candidate selected by SageMaker AutoPilot and create a batch transform job as our test data has more than 100K records.

In [ ]:
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
model_name = best_candidate_name + timestamp_suffix + "-model"
model_arn = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)
print('Model ARN corresponding to the best candidate is : {}'.format(model_arn['ModelArn']))

In [ ]:
transform_job_name = 'automl-covid19-transform-' + timestamp_suffix

transform_input = {
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': test_data_s3_path
            }
        },
        'ContentType': 'text/csv',
        'CompressionType': 'None',
        'SplitType': 'Line'
    }

transform_output = {
        'S3OutputPath': 's3://{}/{}/inference-results'.format(bucket,prefix),
    }

transform_resources = {
        'InstanceType': 'ml.m5.4xlarge',
        'InstanceCount': 1
    }

sm.create_transform_job(TransformJobName = transform_job_name,
                        ModelName = model_name,
                        TransformInput = transform_input,
                        TransformOutput = transform_output,
                        TransformResources = transform_resources
)

Check for Job status

In [ ]:
print ('JobStatus')
print('----------')


describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
job_run_status = describe_response['TransformJobStatus']
print (job_run_status)

while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
    job_run_status = describe_response['TransformJobStatus']
    print (job_run_status)
    sleep(30)

View the results of the transform job.

In [ ]:
s3_output_key = '{}/inference-results/test_data.csv.out'.format(prefix);
local_inference_results_path = 'inference_results.csv'

s3 = boto3.resource('s3')
inference_results_bucket = s3.Bucket(session.default_bucket())

inference_results_bucket.download_file(s3_output_key, local_inference_results_path);

inference_data = pd.read_csv(local_inference_results_path, sep=';', header=None)
#Keep the output on one page
pd.set_option('display.max_rows', 10)
inference_data

If you are performing regression for a continuous value (i.e.linear regression) then you may use metrics such as MSE (mean square error) or RMSE (root mean square error) for evaluating the accuracy of a linear regression model. In our case we will use MSE. AutoPilot has also selected the best model based on MSE.

In [ ]:
MSE = np.square(np.subtract(test_data.reset_index()['risk_infection'],inference_data[0])).mean()
print('Mean Square Error: {0:.2f}'.format(MSE))

Apart from the MSE calculation, visualization in form of charts gives us a better idea about the model. In this case we have ploted the line graphs for the true values vs predicted values. For better readability we have plotted the last 100 values. However keep in mind that the MSE is calculated for the complete test dataset

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 20, 5

plt.plot(test_data.reset_index().tail(100).index.values, test_data.reset_index()['risk_infection'].tail(100), color = "red")
plt.plot(test_data.reset_index().tail(100).index.values, inference_data[0].tail(100), color = "green")
plt.title("True Probability vs Predicted Probability (Mean Square Error:{0:.2f})".format(MSE))
plt.xlabel("Person Index")
plt.ylabel("Infection Risk Probability")
plt.legend(['True Probability','Predicted Probability'])
plt.show()

---
## Cleanup

The Autopilot job creates many underlying artifacts such as dataset splits, preprocessing scripts, or preprocessed data, etc. This code, when un-commented, deletes them. This operation deletes all the generated models and the auto-generated notebooks as well. 

In [ ]:
#s3 = boto3.resource('s3')
#s3_bucket = s3.Bucket(bucket)

#job_outputs_prefix = '{}/output/{}'.format(prefix, auto_ml_job_name)
#s3_bucket.objects.filter(Prefix=job_outputs_prefix).delete()